In [ ]:
import os
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Umbral recomendado para filtrar resultados
# 📌 Basado en experiencias con búsquedas semánticas en embeddings, estos son los rangos recomendados para establecer un umbral de corte:

# Distancia Coseno	Significado	Acción Recomendada
# 0.0 - 0.25	Respuesta muy relevante	✅ Considerar directamente como una respuesta válida
# 0.25 - 0.40	Respuesta posible, pero revisar	⚠️ Puede contener información relevante, pero verificar manualmente
# 0.40 - 0.60	Información posiblemente útil, pero ambigua	❌ Filtrar en caso de que no haya suficiente información relevante
# > 0.60	No es una respuesta relevante	❌ No considerar como respuesta
# 📌 RECOMENDACIÓN:

# Usar un umbral de 0.35 - 0.40 como corte inicial.
# Si no se encuentran suficientes documentos relevantes, aumentar el umbral gradualmente (0.45).
# Si el sistema detecta demasiados documentos irrelevantes, reducir el umbral (0.30).

# Parámetros de filtrado
UMBRAL_BASE = 0.40  # Umbral inicial
UMBRAL_MAXIMO = 0.45  # Umbral máximo en caso de pocos resultados
MIN_DOCUMENTOS_RELEVANTES = 1  # Mínimo de documentos para considerar la respuesta válida

def cargar_embeddings_desde_archivo(ruta_archivo):
    """Carga los embeddings desde un archivo JSON."""
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data["archivo"], data["secciones"]

def calcular_similitud(embedding1, embedding2):
    """Calcula la similitud mediante la distancia del coseno."""
    return cosine(embedding1, embedding2)

def buscar_documentos_similares(texto, carpeta_embeddings):
    """
    Busca documentos en una carpeta que sean más similares al texto de entrada.
    Retorna una lista de tuplas con el nombre del archivo, la menor distancia y el párrafo más similar.
    """
    embedding_texto = embedding_model.encode(texto)
    resultados = {}

    # Recorrer archivos en la carpeta
    for archivo in os.listdir(carpeta_embeddings):
        if archivo.endswith("_embeddings.json"):
            ruta_archivo = os.path.join(carpeta_embeddings, archivo)
            nombre_archivo, secciones = cargar_embeddings_desde_archivo(ruta_archivo)

            # Inicializar menor distancia
            menor_distancia = float("inf")
            parrafo_mas_similar = ""

            for seccion in secciones:
                embedding_parrafo = np.array(seccion["embedding"])
                distancia = calcular_similitud(embedding_texto, embedding_parrafo)

                if distancia < menor_distancia:
                    menor_distancia = distancia
                    parrafo_mas_similar = seccion["texto"]

            resultados[nombre_archivo] = (menor_distancia, parrafo_mas_similar)

    # Ordenar por menor distancia
    resultados_ordenados = sorted(resultados.items(), key=lambda x: x[1][0])

    # Filtrar documentos por umbral de similitud
    umbral_actual = UMBRAL_BASE
    documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    # Si no hay suficientes documentos, ampliar el umbral
    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        umbral_actual = UMBRAL_MAXIMO
        documentos_relevantes = [(archivo, distancia, parrafo) for archivo, (distancia, parrafo) in resultados_ordenados if distancia <= umbral_actual]

    # Si sigue sin haber suficientes documentos relevantes, descartar la búsqueda
    if len(documentos_relevantes) < MIN_DOCUMENTOS_RELEVANTES:
        return []

    return documentos_relevantes

# Ejemplo de uso
carpeta_embeddings = "data"
texto_busqueda = "The cause of food insecurity in Burkina Faso is linked with several factors in addition to drought"

resultados = buscar_documentos_similares(texto_busqueda, carpeta_embeddings)

# Mostrar resultados
if resultados:
    print("\n🔹 Documentos relevantes encontrados:")
    for archivo, distancia, parrafo in resultados:
        print(f"\n📄 {archivo} - Distancia: {distancia:.4f}")
        print(f"📝 Párrafo más similar: {parrafo[:300]}...")  # Mostrar los primeros 300 caracteres
else:
    print("\n⚠️ No se encontró una respuesta clara en los documentos.")





🔹 Documentos relevantes encontrados:

📄 1e8c2332a461a3a142840fa477fa907c66c35dac - Distancia: 0.1083
📝 Párrafo más similar: Effectiveness: Focus on Drought
The cause of food insecurity in Burkina Faso is linked with several factors in addition to drought,
including conflict, agricultural practice, supply chain disruptions and food prices.
● Recommendation: Better justify the selection of rainfall as the sole trigger and ...
